In [1]:
from IPython.display import display, Image, HTML
import ipywidgets as widgets
from ipywidgets import Layout
import ftplib
import pandas as pd

In [2]:
%%html
<style>
.widget-image {
    box-shadow: 5px 5px 19px rgba(0,0,0,0.2)!important;
    padding: 20px!important;
    border-radius: 7px!important;
    max-height: 700px!important;
}
.widget-button {
    background : #6deb6d;
    border-radius: 50px!important;
    letter-spacing: 3px;
    transform: translateX(-50%);
    left: 50%;
    padding: 1.5em;
    height: auto;
    width: 50%;
    box-shadow: 3px 2px 10px rgba(0,0,0,0.1);
    transition: 0.2s;
    margin-bottom: 15px!important;
    
}

.widget-button:hover {
    box-shadow: 3px 2px 15px rgba(0,0,0,0.2)!important;
    transform:  translateX(-50%) scaleX(1.1);
}
#notebook-container {
    width: 80%!important;
}

</style>

In [3]:
data = pd.read_excel('photo_tsx.xlsx')

In [4]:
plu = data.plu_id.astype(str)

In [5]:
first = plu[:643]

In [6]:
second = plu[643:1286]

In [7]:
third = plu[1286:1929]

In [8]:
fourth = plu[1929:]

In [9]:
fourth_1 = fourth[:214]
fourth_2 = fourth[214:428]
fourth_3 = fourth[428:]
len(fourth_1), len(fourth_2), len(fourth_3)

(214, 214, 213)

In [10]:
my_plu = first # меняете на  fourth_2, fourth_3

In [11]:

class rename_photos:
    
    categories_list = ['Выберите категорию',
                        'Лицевая',
                        'Этикетка',
                        'Состав',
                        'Оборотная сторона',
                        'Прочее',
                        'Сверху',
                        'Справа',
                        'Слева',
                        'Снизу',
                        '3 грани']
    
    
    
    def __init__(self, login, password, plu):
        
        self.login = login
        self.password = password
        self.categories = {}
        self.photo_dict = {}
        self.all_plu = [(x[:2], x[2:4], x[4:]) for x in plu]
        self.flag = False
        self.categories_count = []
        
        self.num_pages = len(self.all_plu) // 2 + 1
        self.current_page = 0
        try:
            self.ftp = ftplib.FTP("ftp.x5.ru", user = self.login, passwd = self.password)
        except: 
            print('Какая-то ошибка с подключением к серверу :(')
        
        
        
    def get_widget(self, image, caption, dropdown):
        return widgets.VBox([image, caption, dropdown],
                  layout = Layout(padding = '21px', align_self = 'center'))
    
    def get_dropdown(self):
        return widgets.Dropdown(
                options = self.categories_list,
                layout = Layout(
                    width = '100%', 
                    margin = '10px 0'
                    )
                )
    
    def repeat_rename(self, _):
        self.flag = True
        self.rename(_)
    
    def rename(self, _):

        problem = False
        self.categories_count = dict(zip(self.plu,[[0 for _ in range(len(self.categories_list))] for _ in range(len(self.plu))]))
        if not self.flag:
            for i in self.categories:
                if self.categories[i].value == 'Выберите категорию':
                    l = widgets.Label(f'Для фото {i} не выбрана категория')
                    display(l)
                    problem = True

            if problem:
                repeat_button = widgets.Button(description = 'Все равно переименовать')
                repeat_button.on_click(self.repeat_rename)
                display(repeat_button)
                return
        
        k = 0
        
        while True:
            try:
                print(f"Попытка подключиться {k+1}{(k+1) * '.'}", end = '\r')
                ftp = ftplib.FTP("ftp.x5.ru", user = self.login, passwd = self.password) 
                ftp.cwd('FotoCont_Perek/DBD')
                break
            except TimeoutError:
                k += 1
                continue
            except:
                print('Какая-то ошибка с подключением к серверу :(')
                break
        
        for a, b, c in self.plu:
            
            try:
                ftp.cwd(f'{a}/{b}/{a + b + c}')
            except:
                continue
            photos_list = ftp.nlst()
            for photo in photos_list:
                category_str = self.categories[photo].value
                category_code = self.categories_list.index(category_str)
                new_name = f"{photo.split('_')[0]}_{category_code}.jpeg"
                self.categories_count[(a, b, c)][category_code] += 1
                num_of_categories = self.categories_count[(a, b, c)][category_code]
                if num_of_categories > 1:
                    new_name = f"{photo.split('_')[0]}_{category_code}_{num_of_categories}.jpeg"
                
                ftp.rename(fromname = photo, toname = new_name)
                if new_name != photo:
                    self.categories[new_name] = self.categories[photo]
                    del self.categories[photo]
                
                label = widgets.Label(f'{photo} -> {new_name}')
                display(label)
                
#             print(ftp.nlst())
            ftp.cwd('../../..')
        
        ftp.quit()
        next_button = widgets.Button(description = 'Следующие')
        
        next_button.on_click(self.next_plu)
        display(next_button)
    
    def next_plu(self, _):
#         for w in self.plu_widgets:
#             w.layout.display = 'none'
        from IPython.display import clear_output
        clear_output()
        self.flag = False
        self.current_page += 1
        if self.current_page == self.num_pages:
            display(HTML(f'''<h3 style = 'background:black; 
                     border-radius: 100px; 
                     color: white;
                     padding: 1em; 
                     text-align: center;'>Всё)</h3>'''))
            return
        self.categories = {}
        self.get_photos()
        
    def bef_plu(self, _):
#         for w in self.plu_widgets:
#             w.layout.display = 'none'
        from IPython.display import clear_output
        clear_output()
        self.flag = False
        if self.current_page == 0:
            return
        self.current_page -= 1
        
        self.categories = {}
        self.get_photos()
        
        


        
    
    def get_image_file_ftp(self, photo):
        '''
        Файл передается с ftp сервера маленькими кусками, 
        поэтому делаем функцию, которая будет собирать их
        в единое целое 
        '''
        def callback(x):

            try:
                self.photo_dict[photo] += x

            except KeyError:
                self.photo_dict[photo] = x

        return callback
    
    def get_photos(self):
        
        display(HTML(f'''<h3 style = 'background:yellow; 
                     color: black; 
                     padding: 1em; 
                     text-align: center;'>Страница {self.current_page + 1}/{self.num_pages}</h3>'''))
        nextb = widgets.Button(description = 'Туда ->')
        lastb = widgets.Button(description = '<- Сюда') 
        nextb.on_click(self.next_plu)
        lastb.on_click(self.bef_plu)
        display(widgets.VBox([lastb, nextb]))
        self.plu = self.all_plu[2 * self.current_page: 2 * (self.current_page + 1)] 


        plu_title = lambda x: f"<h3 style = 'background:black; color: white; padding: 1em; text-align: center; border-radius: 100px;'>plu {x}</h3>"
        k = 0
        while True:
            try:
                print(f"Попытка подключиться {k+1}{(k+2) * '.'}", end = '\r')
                ftp = ftplib.FTP("ftp.x5.ru", user = self.login, passwd = self.password) 
                ftp.cwd('FotoCont_Perek/DBD')
                break
            except TimeoutError:
                k += 1
                continue
            except:
                print('Какая-то ошибка с подключением к серверу :(')
                break
        plu_widgets = []
        for a, b, c in self.plu:
            
            
            # переходим по ссылке к фоткам плюхи
            try:
                ftp.cwd(f'{a}/{b}/{a + b + c}')
            except:
                display(HTML(plu_title(f'{a + b + c} не обнаружен')))
                
                continue
            photos_list = ftp.nlst()


            photos = []
            for photo in photos_list:

                # извлекаем фотку
                ftp.retrbinary('RETR %s' % photo, self.get_image_file_ftp(photo))

                dropdown = self.get_dropdown()
                try:
                    index = int(photo.split('.')[0].split('_')[1])
#                     print(index)
                    dropdown.value = self.categories_list[index]
                except: 
                    
                    pass

                self.categories[photo] = dropdown
                image = widgets.Image(value = self.photo_dict[photo])
                caption = widgets.Label(photo)
                photos.append(self.get_widget(image, caption, dropdown))
            plu_widget = widgets.VBox([widgets.HTML(plu_title(a + b + c)),
                                       widgets.HBox(photos, layout = Layout(display = 'grid', 
                                                         grid_template_columns = '1fr 1fr 1fr'))])
            

            display(plu_widget)
            ftp.cwd('../../..')
        ftp.quit()
        button = widgets.Button(description='ПЕРЕИМЕНОВАТЬ')
        button.on_click(self.rename)
        
        display(button)
        
        
    def go(self):
        
        self.plu = self.all_plu[:3]

        self.get_photos()
        

In [15]:
login = "nikita.dushakov" #имя.фамилия
password = "CEwa8043"

In [16]:
a = rename_photos(login = login, password = password, plu = my_plu)

In [17]:
a.go()

Button(description='ПЕРЕИМЕНОВАТЬ', style=ButtonStyle())